In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [6]:
X_np = np.random.rand(100, 4).astype(np.float32)
y_np = np.random.randint(0, 2, size=(100)).astype(np.float32)

X = torch.from_numpy(X_np)
y = torch.from_numpy(y_np)

In [7]:
class RedSimple(nn.Module):
    def __init__(self, n_entradas, n_ocultas, n_salidas):
        super().__init__()  
        # 1. Capa oculta
        self.hidden = nn.Linear(n_entradas, n_ocultas)
        # 2. Capa de salida
        self.output = nn.Linear(n_ocultas, n_salidas)
        # 3. Función de activación ReLU
        self.relu = nn.ReLU()
        # 4. Sigmoide para la salida binaria
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # 5. Paso hacia adelante: entrada → oculta → activación → salida → sigmoide
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.sigmoid(x).squeeze(1)
        return x

# 6. Creamos una instancia: 4 entradas → 8 neuronas ocultas → 1 salida
modelo = RedSimple(n_entradas=4, n_ocultas=8, n_salidas=1)


In [8]:
# 1. Loss: entropía cruzada binaria para entradas de probabilidad
criterion = nn.BCELoss()

# 2. Optimizador Adam con tasa de aprendizaje por defecto
optimizer = optim.Adam(modelo.parameters(), lr=0.001)

In [9]:
n_epochs = 10
batch_size = 10
dataset = torch.utils.data.TensorDataset(X, y)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(n_epochs):
    for X_batch, y_batch in loader:
        # 1. Forward
        y_pred = modelo(X_batch)
        # 2. Calcular pérdida
        loss = criterion(y_pred, y_batch.float())
        # 3. Backprop y optimización
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # 4. Mostrar progreso por época
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}")

Epoch 1/10, Loss: 0.7291
Epoch 2/10, Loss: 0.6936
Epoch 3/10, Loss: 0.7118
Epoch 4/10, Loss: 0.6905
Epoch 5/10, Loss: 0.6644
Epoch 6/10, Loss: 0.6948
Epoch 7/10, Loss: 0.6899
Epoch 8/10, Loss: 0.6865
Epoch 9/10, Loss: 0.6952
Epoch 10/10, Loss: 0.6872


In [10]:
with torch.no_grad():  
    y_prob = modelo(X)             # 1. Predicciones de probabilidad
    y_pred = (y_prob > 0.5).int()  # 2. Convertir a 0/1 con umbral 0.5
    accuracy = (y_pred == y).float().mean()  # 3. Proporción de aciertos
    print(f"Precisión del modelo: {accuracy*100:.2f}%")

Precisión del modelo: 57.00%


In [11]:
print(modelo)  
# Muestra las capas en orden y sus tamaños

RedSimple(
  (hidden): Linear(in_features=4, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [12]:
import ipywidgets as widgets

capas_widget = widgets.IntSlider(min=1, max=5, description='Capas Ocultas:')

def probar_pytorch(cap):
    # 1. Creamos secuencia de capas según slider
    class RedDyn(nn.Module):
        def __init__(self, n_entradas, n_ocultas, n_capas):
            super().__init__()
            self.layers = nn.ModuleList()
            self.layers.append(nn.Linear(n_entradas, n_ocultas))
            for _ in range(n_capas-1):
                self.layers.append(nn.Linear(n_ocultas, n_ocultas))
            self.out = nn.Linear(n_ocultas, 1)
            self.relu = nn.ReLU()
            self.sig = nn.Sigmoid()
        def forward(self, x):
            for layer in self.layers:
                x = self.relu(layer(x))
            x = self.sig(self.out(x)).squeeze(1)
            return x

    model = RedDyn(4, 4, cap)  # 4 características, 4 neuronas por capa
    crit = nn.BCELoss()
    opt = optim.Adam(model.parameters(), lr=0.001)

    # Entrenamiento rápido (3 épocas)
    for _ in range(3):
        y_p = model(X)
        l = crit(y_p, y.float())
        opt.zero_grad(); l.backward(); opt.step()

    # Evaluación
    with torch.no_grad():
        y_p = model(X)
        preds = (y_p>0.5).int()
        acc = (preds==y).float().mean()
    print(f"Precisión con {cap} capas: {acc*100:.2f}%")

widgets.interactive(probar_pytorch, cap=capas_widget)

interactive(children=(IntSlider(value=1, description='Capas Ocultas:', max=5, min=1), Output()), _dom_classes=…